In [1]:
from pathlib import Path
import numpy as np

from PyQt6.QtWidgets import QMainWindow, QWidget, QApplication, QVBoxLayout

import matplotlib
matplotlib.use('Qt5Agg')

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure



In [5]:

path_to_session = Path(r'C:\Users\aaron\FreeMocap_Data\recording_sessions\session_2023-03-20_13_58_15\recording_14_06_10_gmt-4')
path_to_center_of_mass_npy = path_to_session/'output_data'/'center_of_mass'/'total_body_center_of_mass_xyz.npy'
total_body_com_data = np.load(path_to_center_of_mass_npy)


In [3]:


class TimeSeriesPlotCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=15, height=4, dpi=100):
        self.fig = Figure(figsize=(width, height), dpi=dpi)
        self.x_ax = self.fig.add_subplot(311)
        self.y_ax = self.fig.add_subplot(312)
        self.z_ax = self.fig.add_subplot(313)

        super(TimeSeriesPlotCanvas, self).__init__(self.fig)

class TimeSeriesPlotterWidget(QWidget):

    def __init__(self):
        super().__init__()

        self._layout = QVBoxLayout()
        self.setLayout(self._layout)

        self.fig, self.axes_list = self.initialize_skeleton_plot()

        toolbar = NavigationToolbar(self.fig, self)

        self._layout.addWidget(toolbar)
        self._layout.addWidget(self.fig)

    def initialize_skeleton_plot(self):
        fig = TimeSeriesPlotCanvas(self, width=15, height=10, dpi=100)
        self.x_ax = fig.figure.axes[0]
        self.y_ax = fig.figure.axes[1]
        self.z_ax = fig.figure.axes[2]

        self.axes_list = [self.x_ax,self.y_ax,self.z_ax]
        return fig, self.axes_list


    def update_plot(self,freemocap_data:np.ndarray):


        axes_names = ['X Axis (mm)', 'Y Axis (mm)', 'Z Axis (mm)']

        for dimension, (ax,ax_name) in enumerate(zip(self.axes_list,axes_names)):

            ax.cla()
            ax.plot(freemocap_data[:,dimension], label = 'FreeMoCap COM', alpha = .7)

            ax.set_ylabel(ax_name)
            
            if dimension == 2: #put the xlabel only on the last plot
                ax.set_xlabel('Frame #')

            # ax.legend()
        self.fig.fig.suptitle('Center of Mass Position vs. Frame')
        self.fig.fig.figure.canvas.draw_idle()


In [4]:

class CenterOfMassPlotWindow(QMainWindow):
    def __init__(self):
        super().__init__()
    
        layout = QVBoxLayout()
        widget = QWidget()

        self.time_series_viewer = TimeSeriesPlotterWidget()
        layout.addWidget(self.time_series_viewer)

        widget.setLayout(layout)
        self.setCentralWidget(widget)

        self.time_series_viewer.update_plot(freemocap_data=total_body_com_data)


app = QApplication([])
win = CenterOfMassPlotWindow()
win.show()
app.exec()

0